### 3.BERT Fine-tuning for Sentimental Analysis

In [1]:
# 라이브러리 설치
# pip install transformers
# pip install nlp

In [2]:
# 라이브러리
import torch
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset
import numpy as np
import gdown

In [3]:
# 데이터셋 다운로드
url = 'https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc-'
output_path = '/Users/leesanghyuk/Python_Programs/BERT/imdb.csv'
gdown.download(url, output_path, quiet=False)

# 데이터셋 로드
dataset = load_dataset('csv', data_files=output_path, split='train')

Downloading...
From: https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc-
To: /Users/leesanghyuk/Python_Programs/BERT/imdb.csv
100%|██████████| 132k/132k [00:00<00:00, 630kB/s]
Using custom data configuration default


In [4]:
# 데이터 타입 확인
type(dataset)

nlp.arrow_dataset.Dataset

In [5]:
# 데이터셋을 학습/테스트로 분할
dataset = dataset.train_test_split(test_size=0.3)

In [6]:
# 확인
dataset

{'train': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 70),
 'test': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 30)}

In [7]:
# 학습 데이터 생성
train_data = dataset['train']

# 테스트 데이터
test_data = dataset['test']

데이터는 다운 및 분할이 완료되었다. 이제 BERT 모델을 다운 받고 사용하자.

In [8]:
# BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Bert Tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

이제 데이터 전처리를 진행한다. 다음의 단계를 수동으로 해줄 수 있지만, BERT 토크나이저를 사용하면 자동으로 해준다.
- 토큰화
- 특수 토큰 추가
- ID로 변환
- 세그먼트 ID 생성
- 어텐션 마스크 생성

In [10]:
# 토크나이저 사용 예시
tokenizer('I love Paris')

{'input_ids': [101, 1045, 2293, 3000, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [11]:
# 자동으로 패딩을 수행하는 예시
tokenizer(['I love Paris', 'You like cute dog', 'snow fall'], padding=True, max_length=5)

/Users/leesanghyuk/anaconda3/envs/PySang/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'input_ids': [[101, 1045, 2293, 3000, 102, 0], [101, 2017, 2066, 10140, 3899, 102], [101, 4586, 2991, 102, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0]]}

- ```max_length```가 5이기 때문에 토큰 개수가 5개보다 적은 문장은 패딩이 적용되었다.
- 패딩이 진행된 문장은 ```attention_mask```가 1인 부분이 있다.

In [12]:
# 전처리를 위한 함수
def preprocess(data):
    return tokenizer(data['text'], padding=True, truncation=True)

In [13]:
# 에러 해결을 위해 패키지 업데이트
# pip install dill==0.3.5.1.

In [14]:
# 전처리 적용
train_data = train_data.map(preprocess, batched=True, batch_size=len(train_data))
test_data = test_data.map(preprocess, batched=True, batch_size=len(test_data))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

- 계속해서 에러가 발생했다.
- PY3와 관련된 문서는 찾을 수 없어서 비슷한 에러를 언급한 문서를 확인 후 시도해봤더니 해결됐다...😮‍💨
- [스택오버플로우]('https://stackoverflow.com/questions/74857932/attributeerror-module-dill-dill-has-no-attribute-log')를 참고해서 ```dill``` 버전을 조정했다.

In [15]:
# 데이터의 형식 지정
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

이렇게 데이터셋 준비를 완료했다. 모델을 학습시켜보자.

In [30]:
# 하이퍼파라미터 정의
batch_size = 8
epochs = 2

In [31]:
# 웜업, 가중치 디케이 설정
warmup_steps = 500
weight_decay = 0.01

In [32]:
# 학습을 위한 인자 정의
training_args = TrainingArguments(output_dir='./results', 
                                  num_train_epochs=epochs,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  warmup_steps=warmup_steps,
                                  weight_decay=weight_decay,
                                  logging_dir='./logs')

In [33]:
# 트레이너 정의
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=train_data,
                  eval_dataset=test_data)

In [35]:
# 학습
#trainer.train()

In [36]:
# 모델 평가
#trainer.eval()